In [9]:
import os
import pandas as pd
from typing import List
from datetime import datetime
import numpy as np

from lib.utils import get_nyse_date_tups, get_oxford_dfs
import pandas_market_calendars as mcal
import requests
from py_vollib.black_scholes import black_scholes, implied_volatility

In [8]:
df = get_oxford_dfs(1)

In [9]:
df

,c,h,l,n,o,t,v,vw,ticker
0,226.50,226.50,226.50,1,226.50,1539256980000,100,226.5000,TMO
1,227.00,227.00,227.00,2,227.00,1539259740000,150,226.8633,TMO
2,227.00,227.00,227.00,4,227.00,1539259920000,209,226.9957,TMO
3,227.00,227.00,227.00,4,227.00,1539261180000,250,226.8020,TMO
4,228.50,228.50,228.50,4,228.50,1539261360000,214,228.4902,TMO
...,...,...,...,...,...,...,...,...,...
469674,495.75,496.03,495.65,251,496.03,1696881360000,8608,495.8342,TMO
469675,495.85,495.90,495.69,291,495.72,1696881420000,10163,495.7920,TMO
469676,495.93,495.99,495.77,482,495.86,1696881480000,19078,495.8682,TMO
469677,496.09,496.21,495.78,722,495.86,1696881540000,34031,496.0076,TMO


## try manually - non vectorized

```
:param price: the Black-Scholes option price
:param S: underlying asset price
:param K: strike price
:param t: time to expiration in years
:param r: risk-free interest rate
:param flag: 'c' or 'p' for call or put.
```

In [18]:
# List of dictionaries containing mock data
mock_data = [
    {'S': 100.0, 'K': 90.0, 'sigma': 0.2, 't': 1.0, 'r': 0.05, 'flag': 'c'},
    {'S': 120.0, 'K': 100.0, 'sigma': 0.25, 't': 0.5, 'r': 0.03, 'flag': 'c'},
    {'S': 95.0, 'K': 100.0, 'sigma': 0.18, 't': 1.2, 'r': 0.04, 'flag': 'p'},
    {'S': 90.0, 'K': 85.0, 'sigma': 0.22, 't': 2.0, 'r': 0.02, 'flag': 'c'},
    {'S': 110.0, 'K': 100.0, 'sigma': 0.3, 't': 1.5, 'r': 0.05, 'flag': 'p'},
    {'S': 80.0, 'K': 75.0, 'sigma': 0.15, 't': 0.7, 'r': 0.03, 'flag': 'c'},
    {'S': 105.0, 'K': 110.0, 'sigma': 0.28, 't': 0.9, 'r': 0.06, 'flag': 'p'}
]

In [24]:
df = pd.DataFrame(mock_data)

In [58]:
df

,S,K,sigma,t,r,flag
0,100.0,90.0,0.20,1.0,0.05,c
1,120.0,100.0,0.25,0.5,0.03,c
2,95.0,100.0,0.18,1.2,0.04,p
3,90.0,85.0,0.22,2.0,0.02,c
4,110.0,100.0,0.30,1.5,0.05,p
5,80.0,75.0,0.15,0.7,0.03,c
6,105.0,110.0,0.28,0.9,0.06,p


In [62]:
bs_args = ['S', 'K', 't', 'r', 'sigma', 'flag']
iv_args = ['price', 'S', 'K', 't', 'r', 'flag']

In [66]:
# what do i need to get?
# row of data but only if column is in bs_args
# pass that to black scholes
# store it in a new column, 'price'
# pass that to implied volatility
# store it in a new column, 'iv'

df['price'] = df[bs_args].apply(lambda x: black_scholes(**x), axis=1)
df['iv'] = df[iv_args].apply(lambda x: implied_volatility.implied_volatility(**x), axis=1)

In [67]:
df

,S,K,sigma,t,r,flag,price,iv
0,100.0,90.0,0.20,1.0,0.05,c,16.699448,0.20
1,120.0,100.0,0.25,0.5,0.03,c,22.762665,0.25
2,95.0,100.0,0.18,1.2,0.04,p,7.630977,0.18
3,90.0,85.0,0.22,2.0,0.02,c,15.286033,0.22
4,110.0,100.0,0.30,1.5,0.05,p,7.703074,0.30
5,80.0,75.0,0.15,0.7,0.03,c,7.974822,0.15
6,105.0,110.0,0.28,0.9,0.06,p,10.666171,0.28


In [17]:
implied_volatility.implied_volatility(**mock_data[0])

TypeError: implied_volatility() got an unexpected keyword argument 'sigma'

In [26]:
print(*mock_data.values())

c 100 100 0.0547945205479452 0.02 0


In [27]:
iv = oc.implied_volatility(*list(mock_data.values()), **kwargs)

TypeError: 'int' object is not iterable